<a href="https://colab.research.google.com/github/Sanjeev2697UCSD/CDM_Dykstra/blob/main/Antenna_Array_Synthesis_using_CDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import sklearn.datasets as dt
import cvxpy as cp
import csv
import math
from math import e

In [13]:
# Obtaining the gain pattern from -pi/2 to pi/2
a = []
with open('gain_pattern.csv') as csvfile:
  gain_pattern_1 = csv.reader(csvfile)
  i = 0
  for row in gain_pattern_1:
    # Post processing the parameters
    p = row[0]
    p = p.replace('i', 'j')
    p = complex(p)
    a.append(p)

a = np.array(a)

***We are performing Co-ordinate Descent Algorithm for the Antenna Array Synthesis. We try to generate a dolph-chebyshev array based weights that achieves a desired side-lobe level given a gain pattern.***

In [3]:
def coordinate_descent_function(X,y,lambda1, MAX_ITERATIONS):
  # initialize Beta vector = 0
  N = np.shape(y)[0]
  y = y.reshape(N, 1)
  p = np.shape(X[1,])
  beta = np.zeros([p[0],1], dtype=complex)
  num_iterations = 0;
  while(num_iterations <= MAX_ITERATIONS):
    for i in range(p[0]):
      idx_out_columns = [i]
      idx_in_columns = [j for j in range(p[0]) if j not in idx_out_columns]
      X_r = X[:,idx_in_columns]
      beta_r = beta[idx_in_columns,]
      residual = y - np.matmul(X_r, beta_r)[:,]

      # Computing the inner product to figure out how much magnitude of correlation this dimension holds
      # ols_beta = np.matmul(np.transpose(residual) , X[:,idx_out_columns])
      ols_beta = (np.conj(X[:,idx_out_columns].T) @ residual)[0,0]

      normalizer = (np.conj(X[:,idx_out_columns].T) @ X[:,idx_out_columns])[0,0]
      ols_beta = ols_beta/np.round(normalizer,1)
      # lambda2 = lambda1/normalizer

      # Update beta[i] via soft-thresholding
      # beta[i] = compute_soft_thresholding(ols_beta, lambda2)
      beta[i] = ols_beta

    num_iterations = num_iterations + 1

  return beta


def compute_soft_thresholding(ols_beta, lambda1):
  if ols_beta > lambda1:
    return ols_beta + (lambda1)
  elif ols_beta < lambda1:
    return ols_beta - (lambda1)
  else:
    return 0

In [4]:
# Antenna Parameters:

num_antennas = 8
f0 = 77 * 10**9        # 77GHz
lambda1 = (3*10**8)/f0

spacing = lambda1/2;

In [5]:
# Weight vector for debugging
weight_vector = [1,1.97831590137716,3.09652556013193,3.81364267157990,3.81364267157990,3.09652556013193,1.97831590137716,1];
weight_vector = np.round(weight_vector,4)

b = X@weight_vector.T

print(np.shape(b))

NameError: ignored

In [15]:
# Preparing the transformation matrix
num_antennas = 8
num_angles = len(a)
# num_angles = 315

X = np.zeros([num_angles, num_antennas], dtype=complex)
angles = np.arange(-math.pi/2, math.pi/2,0.01)

for i in range(num_angles):
  for k in range(num_antennas):
    X[i,k] = np.round(e**(1j* k * math.sin(angles[i]) * 2*(math.pi/lambda1) * spacing),4)

beta_predicted = np.zeros([num_antennas,1], dtype=complex)

# Fixing lambda1 as zero will perform Co-ordinate descent algorithm without LASSO
beta_predicted = coordinate_descent_function(X,a,0, 100)

beta_predicted = np.round(beta_predicted,4)

In [10]:
print(np.shape(a))

(1,)


In [16]:
print(beta_predicted)


[[1.    +0.j]
 [1.9783+0.j]
 [3.0965+0.j]
 [3.8137+0.j]
 [3.8136+0.j]
 [3.0965+0.j]
 [1.9783+0.j]
 [1.    +0.j]]
